In [1]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import mean_squared_error
from sklearn.ensemble import RandomForestRegressor
from sklearn.tree import DecisionTreeRegressor
import xgboost as xgb
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import GridSearchCV, cross_val_score
from plotly.offline import init_notebook_mode, iplot, plot
import plotly.express as px
from matplotlib import pyplot as plt

In [2]:
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "0"

In [3]:
df1 = pd.read_csv("./Spotify-2000.csv")
print(df1.shape)
print(df1.head())

df2 = pd.read_csv("./top10s.csv", engine='python')
print(df2.shape)
print(df2.head())

(1994, 15)
   Index                   Title             Artist            Top Genre  \
0      1                 Sunrise        Norah Jones      adult standards   
1      2             Black Night        Deep Purple           album rock   
2      3          Clint Eastwood           Gorillaz  alternative hip hop   
3      4           The Pretender       Foo Fighters    alternative metal   
4      5  Waitin' On A Sunny Day  Bruce Springsteen         classic rock   

   Year  Beats Per Minute (BPM)  Energy  Danceability  Loudness (dB)  \
0  2004                     157      30            53            -14   
1  2000                     135      79            50            -11   
2  2001                     168      69            66             -9   
3  2007                     173      96            43             -4   
4  2002                     106      82            58             -5   

   Liveness  Valence Length (Duration)  Acousticness  Speechiness  Popularity  
0        11       6

In [4]:
# 統一兩個dataset的columns(因為順序都一樣)
df2.columns = df1.columns
df2.head()

# 合併兩個資料集成一個
df = df1.append(df2, ignore_index=True)
print(df.shape)
print(df.head())

(2597, 15)
   Index                   Title             Artist            Top Genre  \
0      1                 Sunrise        Norah Jones      adult standards   
1      2             Black Night        Deep Purple           album rock   
2      3          Clint Eastwood           Gorillaz  alternative hip hop   
3      4           The Pretender       Foo Fighters    alternative metal   
4      5  Waitin' On A Sunny Day  Bruce Springsteen         classic rock   

   Year  Beats Per Minute (BPM)  Energy  Danceability  Loudness (dB)  \
0  2004                     157      30            53            -14   
1  2000                     135      79            50            -11   
2  2001                     168      69            66             -9   
3  2007                     173      96            43             -4   
4  2002                     106      82            58             -5   

   Liveness  Valence Length (Duration)  Acousticness  Speechiness  Popularity  
0        11       6

In [5]:
# 丟掉2000年以前的資料
df.drop(df[df.Year < 2000].index, inplace=True)
df.shape

(1402, 15)

In [6]:
df['Top Genre'].value_counts()
unique_genre = df['Top Genre'].unique()
print(unique_genre.shape)
print(unique_genre)   # 原本一共有134種不同的曲風
# print(np.mean(df["Top Genre"].value_counts()))

(134,)
['adult standards' 'album rock' 'alternative hip hop' 'alternative metal'
 'classic rock' 'alternative pop rock' 'pop' 'modern rock'
 'detroit hip hop' 'alternative rock' 'dutch indie' 'garage rock'
 'dutch cabaret' 'permanent wave' 'classic uk pop' 'dance pop'
 'modern folk rock' 'dutch pop' 'dutch americana' 'alternative dance'
 'german pop' 'afropop' 'british soul' 'irish rock' 'disco' 'big room'
 'art rock' 'danish pop rock' 'neo mellow' 'britpop' 'boy band'
 'carnaval limburg' 'arkansas country' 'latin alternative' 'british folk'
 'celtic' 'chanson' 'celtic rock' 'hip pop' 'east coast hip hop'
 'dutch rock' 'blues rock' 'electro' 'australian pop' 'belgian rock'
 'downtempo' 'reggae fusion' 'british invasion' 'finnish metal'
 'canadian pop' 'bow pop' 'dutch hip hop' 'dutch metal' 'soft rock'
 'acoustic pop' 'acid jazz' 'dutch prog' 'candy pop' 'operatic pop'
 'trance' 'scottish singer-songwriter' 'mellow gold' 'alternative pop'
 'dance rock' 'atl hip hop' 'eurodance' 'blues'

In [7]:
# reset index
print(df.shape)
df.reset_index(inplace=True)
df.shape

(1402, 15)


(1402, 16)

In [8]:
for i in range(0, len(df['Top Genre'])):
    if(df['Top Genre'][i].find("pop") != -1):
        df['Top Genre'][i] = "pop"
    elif(df['Top Genre'][i].find("hip hop") != -1):
        df['Top Genre'][i] = "hip hop"
    elif(df['Top Genre'][i].find("r&b") != -1):
        df['Top Genre'][i] = "r&b"
    elif(df['Top Genre'][i].find("folk") != -1):
        df['Top Genre'][i] = "folk"
    elif(df['Top Genre'][i].find("metal") != -1):
        df['Top Genre'][i] = "metal"
    elif(df['Top Genre'][i].find("jazz") != -1):
        df['Top Genre'][i] = "jazz"
    elif(df['Top Genre'][i].find("rock") != -1):
        df['Top Genre'][i] = "rock"
    elif(df['Top Genre'][i].find("edm") != -1):
        df['Top Genre'][i] = "edm"
unique_genre = df['Top Genre'].unique()

C:\Users\user\Anaconda3\envs\pytorch_p36\lib\site-packages\ipykernel_launcher.py:15: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

C:\Users\user\Anaconda3\envs\pytorch_p36\lib\site-packages\ipykernel_launcher.py:5: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

C:\Users\user\Anaconda3\envs\pytorch_p36\lib\site-packages\ipykernel_launcher.py:11: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

C:\Users\user\Anaconda3\envs\pytorch_p36\lib\si

In [9]:
print(df['Top Genre'].value_counts()[:10])
print(np.mean(df['Top Genre'].value_counts()))  # 21

pop                724
rock               186
dutch indie         52
metal               42
hip hop             40
british soul        36
permanent wave      33
dutch cabaret       27
neo mellow          26
adult standards     21
Name: Top Genre, dtype: int64
21.242424242424242


In [10]:
# 只留下前10大類，剩餘小類當作others
top10_genre = list(df['Top Genre'].value_counts()[:10].index)
print(df['Top Genre'][:10])
for i in range(0, len(df['Top Genre'])):
    if(df['Top Genre'][i] not in top10_genre):
        df['Top Genre'][i] = "others"
# df.loc[df['Top Genre'] not in top10_genre, 'Top Genre'] = 'others'
print(df['Top Genre'][:10])

0    adult standards
1               rock
2            hip hop
3              metal
4               rock
5                pop
6                pop
7               rock
8               rock
9            hip hop
Name: Top Genre, dtype: object
0    adult standards
1               rock
2            hip hop
3              metal
4               rock
5                pop
6                pop
7               rock
8               rock
9            hip hop
Name: Top Genre, dtype: object


C:\Users\user\Anaconda3\envs\pytorch_p36\lib\site-packages\ipykernel_launcher.py:6: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [11]:
print(df['Top Genre'].value_counts())

pop                724
others             215
rock               186
dutch indie         52
metal               42
hip hop             40
british soul        36
permanent wave      33
dutch cabaret       27
neo mellow          26
adult standards     21
Name: Top Genre, dtype: int64


In [12]:
print(df.shape)
df.head()

(1402, 16)


,index,Index,Title,Artist,Top Genre,Year,Beats Per Minute (BPM),Energy,Danceability,Loudness (dB),Liveness,Valence,Length (Duration),Acousticness,Speechiness,Popularity
0,0,1,Sunrise,Norah Jones,adult standards,2004,157,30,53,-14,11,68,201,94,3,71
1,1,2,Black Night,Deep Purple,rock,2000,135,79,50,-11,17,81,207,17,7,39
2,2,3,Clint Eastwood,Gorillaz,hip hop,2001,168,69,66,-9,7,52,341,2,17,69
3,3,4,The Pretender,Foo Fighters,metal,2007,173,96,43,-4,3,37,269,0,4,76
4,4,5,Waitin' On A Sunny Day,Bruce Springsteen,rock,2002,106,82,58,-5,10,87,256,1,3,59


In [13]:
# 去除不必要attribute
df.drop(columns=['index', 'Index', 'Title'], inplace=True)
print(df.shape)
df.head()

(1402, 13)


,Artist,Top Genre,Year,Beats Per Minute (BPM),Energy,Danceability,Loudness (dB),Liveness,Valence,Length (Duration),Acousticness,Speechiness,Popularity
0,Norah Jones,adult standards,2004,157,30,53,-14,11,68,201,94,3,71
1,Deep Purple,rock,2000,135,79,50,-11,17,81,207,17,7,39
2,Gorillaz,hip hop,2001,168,69,66,-9,7,52,341,2,17,69
3,Foo Fighters,metal,2007,173,96,43,-4,3,37,269,0,4,76
4,Bruce Springsteen,rock,2002,106,82,58,-5,10,87,256,1,3,59


In [15]:
df.drop(columns=['Artist', 'Top Genre'], inplace=True)
print(df.shape)
df.head()

(1402, 11)


,Year,Beats Per Minute (BPM),Energy,Danceability,Loudness (dB),Liveness,Valence,Length (Duration),Acousticness,Speechiness,Popularity
0,2004,157,30,53,-14,11,68,201,94,3,71
1,2000,135,79,50,-11,17,81,207,17,7,39
2,2001,168,69,66,-9,7,52,341,2,17,69
3,2007,173,96,43,-4,3,37,269,0,4,76
4,2002,106,82,58,-5,10,87,256,1,3,59


In [17]:
X = df.drop(columns=['Popularity'])
y = df["Popularity"]

In [18]:
X_train, X_test, y_train, y_test = train_test_split(X,
                                                    y,
                                                    test_size=.2,
                                                    random_state=3)

In [19]:
scaler = StandardScaler()

# 計算 X_train 的平均值與變異數
scaler.fit(X_train)

# 標準化 X_train, X_test => X_train_scale, X_test_scale
X_train_scale = scaler.transform(X_train)   
X_test_scale = scaler.transform(X_test)

In [30]:
from sklearn.model_selection import KFold

# 使用grid search找出 XGBRegressor 最佳參數 
# param_grid = {'n_estimators': [100, 200], 'max_depth': [2,3], 'learning_rate': [0.01, 0.05]}
# reg = GridSearchCV(xgb.XGBRegressor(), param_grid, scoring="neg_mean_squared_error", cv=3)
reg = xgb.XGBRegressor(n_estimators=100)  # original model
reg.fit(X_train_scale, y_train)

predicted_XGBoost = reg.predict(X_test_scale)
print("Mean Absolute Error using XGBoost: ", mean_absolute_error(y_test, predicted_XGBoost))
scores = cross_val_score(reg, X, y, scoring='r2') 
print(scores)

Mean Absolute Error using XGBoost:  12.650992213619137
[nan nan nan nan nan]


C:\Users\user\Anaconda3\envs\pytorch_p36\lib\site-packages\sklearn\model_selection\_validation.py:619: FitFailedWarning:

Estimator fit failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "C:\Users\user\Anaconda3\envs\pytorch_p36\lib\site-packages\sklearn\model_selection\_validation.py", line 598, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "C:\Users\user\Anaconda3\envs\pytorch_p36\lib\site-packages\xgboost\core.py", line 506, in inner_f
    return f(**kwargs)
  File "C:\Users\user\Anaconda3\envs\pytorch_p36\lib\site-packages\xgboost\sklearn.py", line 776, in fit
    enable_categorical=self.enable_categorical,
  File "C:\Users\user\Anaconda3\envs\pytorch_p36\lib\site-packages\xgboost\sklearn.py", line 295, in _wrap_evaluation_matrices
    enable_categorical=enable_categorical,
  File "C:\Users\user\Anaconda3\envs\pytorch_p36\lib\site-packages\xgboost\sklearn.py", line 

In [22]:
forest_model = RandomForestRegressor(n_estimators=100, max_depth=10)
forest_model.fit(X_train, y_train)
predicted_random_forest = forest_model.predict(X_test)
print("Mean Absolute Error using Random Forest:", mean_absolute_error(y_test, predicted_random_forest))

Mean Absolute Error using Random Forest: 12.027647127093509


In [23]:
print(predicted_XGBoost[:5])
print(predicted_random_forest[:5])   # 目測random forest有點在亂猜61附近的值(可能是population的平均值)
print(y_test[:5])

[55.012825 58.923534 52.79387  65.43667  62.20118 ]
[64.89213034 58.23335119 53.31652906 67.64771991 54.79816227]
718    31
85     42
540    72
707    59
199    59
Name: Popularity, dtype: int64
